In [42]:
import numpy as np
from pathlib import Path
from tqdm import tqdm
from scipy.io import loadmat
import random
import math
from numpy import genfromtxt
import resampy
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
# this is not where the data is, but where we will save the process
data_root = Path("/data/stg60/new_realdata/version_1/")
data_milton_par = Path("/data/stg60/new_realdata/MoreMilton/")
data_milton_pred = Path("/data/stg60/new_realdata/moreMIltonTest/")

In [3]:
def read_tsv(tsv, start_index = 1024, end_index = None, scale = True):
    dat = genfromtxt(tsv, delimiter='\t')
    x1,x2,x3,x4,x5,x6 = np.hsplit(dat,6)
    #ell_1=((x1-x4)**2+(x2-x5)**2+(x3-x6)**2)**0.5 + 1e-10
    ell_1=((x1-x4)**2+(x3-x6)**2)**0.5 + 1e-10
    ang_sin_x_1=(x1-x4)/ell_1
    phi =np.arcsin(ang_sin_x_1)*180/np.pi
    phi = phi.reshape(-1)[start_index:end_index]
    x1 = x4.reshape(-1)[start_index:end_index]
    phi = resampy.resample(phi, sr_orig = 250, sr_new = 100)
    x1 = resampy.resample(x1, sr_orig = 250, sr_new = 100)
    if scale:
        #phi = phi/20
        phi = phi/1
        #x1 = x1/0.335
        x1 = x1/1
    return phi, x1

In [14]:
data_path_list = sorted(data_milton_par.glob('*.tsv'))

In [15]:
len(data_path_list)

24

In [26]:
phisum=[]
x1sum=[]
for x in tqdm(data_path_list):
    phi,x1 = read_tsv(x)
    phisum.append(phi)
    x1sum.append(x1)

100%|██████████| 24/24 [00:00<00:00, 25.09it/s]


In [39]:
concatphi = np.concatenate(phisum)
concatx1 = np.concatenate(x1sum)

In [38]:
concatphi.shape

(58988,)

In [40]:
concatx1.shape

(58988,)

In [41]:
data = {'phi1': concatphi,
        'x1': concatx1,
        }

In [43]:
df = pd.DataFrame(data)

In [44]:
df.head()

,phi1,x1
0,1.439641,-38.407676
1,1.783142,-55.564236
2,0.892843,-41.750713
3,0.522156,-40.698079
4,-0.124227,-31.621018


In [ ]:
df.to_csv('output_real.csv', index=False)